In [5]:
import torch
import sys

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"Device: {'CUDA' if torch.cuda.is_available() else 'MPS' if torch.backends.mps.is_available() else 'CPU'}")

Python: 3.11.14 | packaged by conda-forge | (main, Oct 22 2025, 22:56:31) [Clang 19.1.7 ]
PyTorch: 2.5.1
Device: MPS


In [6]:
import torch
import time
import json
import shutil
import matplotlib.pyplot as plt
from ultralytics import YOLO # API for loading YOLO models
from thop import profile # Calculates FLOP's & parameter count
from pathlib import Path

# device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
device='cpu'
print(f"Using device: {device}")

Using device: cpu


# YOLOv8 Training

Train YOLOv8n model on the dental X-ray dataset.


In [7]:
# Setup paths
project_root = Path('..').resolve() if Path.cwd().name == 'notebooks' else Path('.').resolve()
processed = project_root / 'data' / 'processed'
models_dir = project_root / 'models'
models_dir.mkdir(parents=True, exist_ok=True)

# Locate dataset
data_yaml_path = processed / 'dataset.yaml'
if not data_yaml_path.exists():
    raise FileNotFoundError("Dataset not found! Run 01_prepare_dataset.ipynb first.")

data_yaml_path = str(data_yaml_path)
test_images_path = Path(data_yaml_path).parent / "test" / "images"

print(f"✓ Data config: {data_yaml_path}")
print(f"✓ Models dir: {models_dir}")

✓ Data config: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/data/processed/dataset.yaml
✓ Models dir: /Users/theatulgupta/Desktop/Deep Learning Project/dental-xray-cavity-detection/models


## Train Model


In [ ]:
# Loading nano-size YOLOv8 model
model_v8 = YOLO('yolov8n.pt')

print("Training YOLOv8n...")

# Start timestamp
start_time_v8 = time.time()

# Begins training
results_v8 = model_v8.train(
    data=data_yaml_path,
    epochs=100,
    imgsz=640, # Default for YOLO
    batch=8,
    name='yolov8n_dental', # Creates folder -> runs/train/
    device=device,
    verbose=True,  # Progress bar
    plots=False,   # Disable auto plot generation
    save=True # Ensures checkpoint
)

# Time required for training
train_time_v8 = time.time() - start_time_v8

path_v8_best_weights = results_v8.save_dir / 'weights' / 'best.pt'
print(f"\n✓ Training complete: {train_time_v8:.2f}s")
print(f"✓ Best weights: {path_v8_best_weights}")

Training YOLOv8n...
Ultralytics YOLOv8.1.34 🚀 Python-3.11.14 torch-2.5.1 CPU (Apple M4)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/data/processed/dataset.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, val_period=1, cache=False, device=cpu, workers=8, project=None, name=yolov8n_dental2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

/Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/yolov12/ultralytics/engine/trainer.py:276: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning /Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/data/processed/train/labels.cache... 51 images, 0 backgrounds, 0 corrupt: 100%|██████████| 51/51 [00:00<?, ?it/s]
val: Scanning /Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/data/processed/val/labels.cache... 6 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6/6 [00:00<?, ?it/s]

optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/yolov12/runs/detect/yolov8n_dental2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size



  0%|          | 0/7 [00:00<?, ?it/s]/Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/yolov12/ultralytics/engine/trainer.py:379: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self.amp):
      1/100         0G      2.029      4.258      1.569        213        640: 100%|██████████| 7/7 [00:07<00:00,  1.08s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.46it/s]

                   all          6        172          0          0          0          0

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all          6        172      0.952      0.932      0.961      0.664

100 epochs completed in 0.236 hours.
Optimizer stripped from /Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/yolov12/runs/detect/yolov8n_dental2/weights/last.pt, 6.2MB



/Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/yolov12/ultralytics/utils/torch_utils.py:486: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x = t

Optimizer stripped from /Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/yolov12/runs/detect/yolov8n_dental2/weights/best.pt, 6.2MB

Validating /Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/yolov12/runs/detect/yolov8n_dental2/weights/best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.11.14 torch-2.5.1 CPU (Apple M4)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


/Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/yolov12/ultralytics/nn/tasks.py:733: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.lo

                   all          6        172      0.973      0.929      0.969      0.679
        cavity_class_0          6         55      0.981      0.943      0.978      0.741
        cavity_class_1          6         45       0.94      0.933      0.959      0.674
        cavity_class_2          6         24       0.99      0.875      0.963      0.677
        cavity_class_3          6         48      0.979      0.967      0.975      0.623
Speed: 0.4ms preprocess, 21.8ms inference, 0.0ms loss, 2.0ms postprocess per image

✓ Training complete: 850.39s
✓ Best weights: /Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/yolov12/runs/detect/yolov8n_dental2/weights/best.pt


## Profiling Model

Performance check-up of your neural network


In [8]:
print("Profiling model...")
# Reloads trained model from saved weights
model_v8_trained = YOLO(path_v8_best_weights)

# Move to CPU for THOP compatibility
model_cpu = model_v8_trained.model.cpu()

# (batch_size, channels, height, width)
dummy_input = torch.randn(1, 3, 640, 640) # For simulation

# No. of parameters(weights), no of operations
flops_v8, params_v8 = profile(model_cpu, inputs=(dummy_input,), verbose=False)
print(f"✓ Parameters: {params_v8/1e6:.2f}M | FLOPs: {flops_v8/1e9:.2f}G")

Profiling model...


NameError: name 'path_v8_best_weights' is not defined

## Measure Inference Time


In [ ]:
print("Measuring inference time...")

# Reload model on correct device for inference
model_v8_trained = YOLO(path_v8_best_weights)

# Collects test images
test_images = list(test_images_path.glob('*.jpg')) + list(test_images_path.glob('*.png'))

def get_avg_inference_time(model, image_list):
    if len(image_list) == 0: # Node images
        return 0
    _ = model(image_list[0], verbose=False)  # Warm-up -> avoids lazy init
    total_time = 0
    for img in image_list:
        start = time.perf_counter()
        _ = model(img, verbose=False)
        total_time += time.perf_counter() - start
    return (total_time / len(image_list)) * 1000

avg_inf_time_v8 = get_avg_inference_time(model_v8_trained, test_images[:10])
print(f"✓ Avg inference: {avg_inf_time_v8:.2f}ms/image ({len(test_images)} test images)")

Measuring inference time...
✓ Avg inference: 78.44ms/image (7 test images)


In [4]:
print("Evaluating on test set...")

# Runs Ultralytics validation on the test split
metrics_v8 = model_v8_trained.val(split='test', data=data_yaml_path, verbose=False)
map50_v8 = metrics_v8.box.map50
map50_95_v8 = metrics_v8.box.map

print(f"✓ mAP@50: {map50_v8:.4f} | mAP@50-95: {map50_95_v8:.4f}")

Evaluating on test set...


NameError: name 'model_v8_trained' is not defined

## Save Results


In [3]:
print("Saving model and metrics...")
shutil.copy(path_v8_best_weights, models_dir / 'yolov8_best.pt')

# Metrics Dictionary
metrics_data = {
    'model': 'YOLOv8n',
    'training_time': train_time_v8,
    'params': params_v8 / 1e6,
    'inference_time_ms': avg_inf_time_v8,
    'map50': float(map50_v8),
    'map50_95': float(map50_95_v8)
}

# Write matrics to JSON
with open(models_dir / 'yolov8_metrics.json', 'w') as f:
    json.dump(metrics_data, f, indent=2)

print(f"\n{'='*60}")
print(f"✓ Model saved: {models_dir}/yolov8_best.pt")
print(f"✓ Metrics saved: {models_dir}/yolov8_metrics.json")
print(f"{'='*60}")

print("\nYOLOv8 training complete!")

Saving model and metrics...


NameError: name 'path_v8_best_weights' is not defined